In [1]:
import numpy as np
import numpy.random as random
import pandas as pd
import scanpy as sc
import anndata

import torch
from scvi.dataset.dataset import GeneExpressionDataset
from scvi.inference import UnsupervisedTrainer, load_posterior
from scvi.models import SCANVI, VAE
import scipy.io as sio
from scipy.sparse import coo_matrix
import os
from umap import UMAP

PRJ_DIR = "/home/derek/research/Kim-Lab/normalization-simulation/"
IN_DIR = PRJ_DIR + "data/pbmc-33k/filtered_gene_bc_matrices/hg19/"
OUT_DIR = PRJ_DIR + "exp/exp-12/out/"

/home/derek/miniconda3/envs/py-ds-scanpy-2/lib/python3.8/site-packages/anndata/core/anndata.py:17: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


In [2]:
cell_type_list = ["CD4_Memory", "CD4_Naive", "Mono_CD14", "B_Pre",
                  "CD8_Memory", "CD8_Naive", "NK_Dim", "Mono_CD16",
                  "CD8_Effector", "B_Pro", "DC", "NK_Bright", "Mk", "pDC"]
cell_type_idx = 10
# bayes = 2.3
bayes = 3

In [3]:
file_prefix = cell_type_list[cell_type_idx]
print(f'working on {file_prefix}')
file_name = OUT_DIR + file_prefix + '-cm.mm'
counts_sparse = sio.mmread(file_name)
counts = counts_sparse.toarray().astype('float64')

file_name = OUT_DIR + file_prefix + '-ident.csv'
batch_list = pd.read_csv(file_name)

working on DC


In [4]:
from scvi.dataset import AnnDatasetFromAnnData
adata = anndata.AnnData(X=counts.T)
adata.obs['group'] = batch_list['x'].to_numpy()

adata.obs['group'] = adata.obs['group'].replace({'group1': 0, 'group2': 1})
adata.obs['group'] = adata.obs['group'].astype(int)
sc.pp.filter_genes(adata, min_cells=5)

In [5]:
cm = AnnDatasetFromAnnData(adata)
cm

[2020-07-11 15:06:24,128] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-07-11 15:06:24,129] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-07-11 15:06:24,143] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-11 15:06:24,145] INFO - scvi.dataset.dataset | Downsampled from 204 to 204 cells


GeneExpressionDataset object with n_cells x nb_genes = 204 x 6265
    gene_attribute_names: 'gene_names', 'n_cells'
    cell_attribute_names: 'batch_indices', 'labels', 'group', 'local_means', 'local_vars'
    cell_categorical_attribute_names: 'batch_indices', 'labels'

In [6]:
n_epochs = 400
n_de = 4000

In [7]:
cm.subsample_genes(n_de)

[2020-07-11 15:06:24,156] INFO - scvi.dataset.dataset | extracting highly variable genes using seurat_v3 flavor


Transforming to str index.


[2020-07-11 15:06:24,233] INFO - scvi.dataset.dataset | Downsampling from 6265 to 4000 genes
[2020-07-11 15:06:24,255] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-11 15:06:24,261] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-07-11 15:06:24,284] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-11 15:06:24,287] INFO - scvi.dataset.dataset | Downsampled from 204 to 204 cells


In [8]:
# loading step
use_cuda = True
vae = VAE(cm.nb_genes, n_batch=0, n_labels=0,
          n_hidden=128, n_layers=1, n_latent=10, dispersion='gene')
file_name = f'8-{file_prefix}-full_posterior'
save_dir = os.path.join(OUT_DIR, file_name)
print(f'Reading from {save_dir}...')
full = load_posterior(dir_path=save_dir,
                      model=vae,
                      use_cuda=use_cuda,
                      )

Reading from /home/derek/research/Kim-Lab/normalization-simulation/exp/exp-12/out/8-DC-full_posterior...
[2020-07-11 15:06:24,366] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-07-11 15:06:24,366] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-07-11 15:06:24,374] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-07-11 15:06:24,375] INFO - scvi.dataset.dataset | Downsampled from 204 to 204 cells


In [9]:
idx1 = adata.obs['group'] == 0
idx2 = adata.obs['group'] == 1
n_samples = 100
M_permutation = 100000

n_de = 0
for i in range(30):
    marker = full.differential_expression_score(idx1, idx2,
                                                n_samples=n_samples,
                                                M_permutation=M_permutation)
    n_de = n_de + np.sum(np.logical_or(marker['bayes_factor'] > bayes,
                                       marker['bayes_factor'] < -bayes))

res = int(n_de/30) if (n_de % 30 < 0.5) else int(n_de/30) + 1
res

1